# Mini Project: Flask for ML Tasks

Flask is a micro web framework for Python that simplifies the process of building web applications. It's designed to be lightweight, flexible, and easy to use, making it a popular choice for developing web applications, APIs, and microservices. Flask doesn't impose a lot of constraints on how you structure your application, giving you the freedom to design your application as needed.

Here's how Flask works and how it can be used for machine learning engineering:

**1. Web Application Basics:**
Flask allows you to define routes (URLs) and associate them with Python functions. When a user visits a specific URL, the associated function is executed, and you can return HTML, JSON, images, or any other type of data as a response. This makes it great for building dynamic and interactive web applications.

**2. Integration with Machine Learning:**
Flask can be incredibly useful in the context of machine learning engineering for a variety of purposes:

- **Model Deployment:** Flask can be used to deploy machine learning models as APIs. Once your model is trained, you can create a Flask app that exposes the model through a web API. This enables other applications, platforms, or services to easily interact with and use your trained models for making predictions.

- **Data Visualization:** You can use Flask to create interactive visualizations of data and model predictions. This is particularly useful when you want to provide users with a graphical representation of your machine learning insights.

- **Custom Preprocessing:** If your machine learning model requires preprocessing steps before making predictions, Flask can be used to create an interface where users can input data, and your app preprocesses the data and feeds it to the model for prediction.

- **User Interface:** Flask can help you create user-friendly interfaces for your machine learning models, allowing non-technical users to interact with and utilize your models without writing code.

- **A/B Testing and Experimentation:** Flask apps can be used to deploy different versions of your models for A/B testing or experimentation, allowing you to compare the performance of different models and make data-driven decisions.

**3. Real-world Example:**
Imagine you've trained a sentiment analysis model, and you want to allow users to input text and get predictions about the sentiment of the text. You can create a Flask app that provides a simple web interface where users can type in their text, and the app sends the text to your sentiment analysis model and displays the result.

A typical Flask application follows a specific structure that helps organize your code, templates, static files, and other resources. This structure is designed to keep your codebase organized and maintainable as your application grows. The layout typically consists of several key components:

1. **Main Application File (`app.py` or similar):**
   This is the entry point of your Flask application. It contains the core configuration, route definitions, and application setup. You define the routes (URLs) your application will respond to, and associate each route with a Python function that handles the logic for that route.

2. **Templates Folder (`templates/`):**
   This folder contains your HTML templates. Flask uses a templating engine (Jinja2 by default) to generate dynamic HTML content. Templates allow you to separate the presentation (HTML) from the logic (Python) in your application, making your code more organized and maintainable. Templates can also include placeholders for dynamic data that Flask fills in when rendering the template.

3. **Static Folder (`static/`):**
   The `static` folder holds static assets like CSS files, JavaScript files, images, fonts, and other resources that are directly served to the user's browser. These assets are typically used to style and enhance the appearance of your web application. By separating static assets from dynamic content, you can optimize the performance of your application.

4. **Other Application Files and Modules:**
   Depending on the complexity of your application, you might create additional modules or packages to handle different aspects of your application, such as database interactions, machine learning model deployment, authentication, and more. These modules help keep your codebase modular and maintainable.

Here's a simplified example of the structure of a Flask application:

```
my_flask_app/
├── app.py
├── templates/
│   ├── index.html
│   ├── result.html
├── static/
│   ├── styles.css
│   ├── script.js
└── other_modules/
    ├── database.py
    ├── ml_model.py
```

In this example:

- `app.py` is the main application file where you define routes, configure the app, and handle requests.
- The `templates` folder contains HTML templates that define the structure of your web pages.
- The `static` folder holds static assets like CSS and JavaScript files.
- The `other_modules` folder might contain additional modules or packages for handling specific tasks, like interacting with a database (`database.py`) or deploying a machine learning model (`ml_model.py`).

The separation of static files and templates helps you maintain clean and organized code. Static assets are served directly to users, while templates are used to dynamically generate HTML pages. This structure facilitates collaboration among team members and makes it easier to understand, maintain, and extend your Flask application.

In this mini-project you'll be introduced to Flask by developing an application where your render a bar plot based on user-input data through an API. Note, colab is not a natural place for API development. Colab is mainly used here to be consistent with other mini-projects.

As such, we'll need to leverage a tool called [ngrok](https://ngrok.com/). Ngrok is a tool that allows you to create secure tunnels from a public endpoint to a locally running web service or application. It enables you to expose your local server to the internet, making it accessible to others even if it's running on your own computer behind firewalls or NAT (Network Address Translation) devices. In this mini-project, we'll use ngrok to expose your colab instance to the wider internet. Again, you'd never use something like this for production.

Go ahead and sign up for an ngrok account [here](https://ngrok.com/). It's free and useful for a wide variety of tasks.

In [2]:
!pip install pyngrok
!pip install flask

Execute the cell below to authenticate to ngrok. Paste your authentication token after copying it from [https://dashboard.ngrok.com/auth](https://dashboard.ngrok.com/auth). This assumes you've already created an ngrok account.  

In [15]:
import getpass
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


In [16]:
import os

os.makedirs('static', exist_ok=True)
os.makedirs('templates', exist_ok=True)

In [17]:
css_content = """.chart-container {
    width: 80%;
    margin: 20px auto;
}
"""

with open('static/styles.css', 'w') as css_file:
    css_file.write(css_content)

In [18]:
html_content = """<!DOCTYPE html>
<html>
<head>
    <title>Data Visualization with Flask</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <h1>Data Visualization with Flask</h1>
    <form method="POST" action="/">
        <div>
            <label for="categories">Categories (comma-separated):</label>
            <input type="text" name="categories" required>
        </div>
        <div>
            <label for="values">Values (comma-separated):</label>
            <input type="text" name="values" required>
        </div>
        <button type="submit">Generate Bar Chart</button>
    </form>
    <div class="chart-container">
        {% if chart_url %}
            <img src="data:image/png;base64,{{ chart_url }}" alt="Bar Chart">
        {% endif %}
    </div>
</body>
</html>"""

with open('templates/index.html', 'w') as html_file:
    html_file.write(html_content)

Time to create our Flask application. Here are your tasks:

1. In your colab notebook create a new folder named `static`. In the folder create a file called `styles.css` and populate it with the following code:
```css
.chart-container {
    width: 80%;
    margin: 20px auto;
}
```
This creates parameters for the bar chart we'll be creating. Note, the full directory of the file should be `/content/static/styles.css`.
2. Create another folder named `templates`. Create a file called `index.html` and populate it with the following html code:
```html
<!DOCTYPE html>
<html>
<head>
    <title>Data Visualization with Flask</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <h1>Data Visualization with Flask</h1>
    <form method="POST" action="/">
        <div>
            <label for="categories">Categories (comma-separated):</label>
            <input type="text" name="categories" required>
        </div>
        <div>
            <label for="values">Values (comma-separated):</label>
            <input type="text" name="values" required>
        </div>
        <button type="submit">Generate Bar Chart</button>
    </form>
    <div class="chart-container">
        {% if chart_url %}
            <img src="data:image/png;base64,{{ chart_url }}" alt="Bar Chart">
        {% endif %}
    </div>
</body>
</html>
```
Html is a pretty intuitive language to understand. Go through each line and make sure you understand what's happening. An extremely useful skill for an engineer is to be able to *understand* any kind of code, even if the engineer doesn't fully know how to write in the language.

1. Write a function called `generate_bar_chart` that takes a list of category names and respective values and generates a bar chart using [plt.bar](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html). In the function, return your barplot as a UTF-8 encoded string. It may be useful to create a BytesIO object using [io.BytesIO](https://docs.python.org/3/library/io.html#io.BytesIO) to store the barplot and then [base64.b64encode](https://docs.python.org/3/library/base64.html#base64.b64encode) to create the encoding.
2. Now, you'll be filling in the `index` function that serves as the landing page for our application. If you're not too familiar with how HTTP works, read this quick primer [here](https://jaimelightfoot.com/blog/how-http-works/). From the form defined in `index.html`, extract the input for the `categories` field and store it in a list. Also, extract the input for the `values` field and store it in a list as well.
3. Pass the two lists from the previous step to your function `generate_bar_chart`.
4. The output of `generate_bar_chart` in the index function should then be passed to Flask's [render_template](https://flask.palletsprojects.com/en/2.3.x/api/#flask.render_template) and returned by the function.
5. Execute the cell below with your code and click on the link output by "ngrok tunnel". This is the application landing page. See if everything works.
6. Have fun with the application built here. See if you can extend it in some way. Make sure you understand how the python code interacts with the html template.

**If you rerun the cell below and get a port issue, go ahead and restart the colab runtime environment and try again.**

In [19]:
from flask import Flask, render_template, request
import threading
import matplotlib.pyplot as plt
import io
import os
import base64

os.environ["FLASK_DEBUG"] = "development"

app = Flask(__name__)
port = 5001

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

def generate_bar_chart(categories, values):
    # Write code here for a fuction that takes a list of category names and
    # respective values and generates a bar chart using plt.bar. Return your
    # barplot as a UTF-8 encoded string.
  plt.bar(categories, values)
  plt.xlabel('Category')
  plt.ylabel('Values')
  plt.title('Bar Chart')

  buffer = io.BytesIO()
  plt.savefig(buffer, format='png')
  plt.close()

  image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

  return image_base64

@app.route('/', methods=['GET', 'POST'])
def index():
    chart_url = None

    if request.method == 'POST':
        # Extract categories from the request form and convert the string to a
        # list.
        categories = request.form.get("categories", "")
        categories = categories.split(",")

        # Extract values from the request form and convert the input string to a
        # list of integers.
        values = request.form.get("values", "")
        values = values.split(",")

        # Pass your categories and values to the generate_bar_chart function.
        chart_url = generate_bar_chart(categories, values)

    # Return a render_template function, passing your bar plot as input.
    return render_template('index.html', chart_url=chart_url)

if __name__ == '__main__':
    # Start the Flask server in a new thread
  threading.Thread(target=app.run, kwargs={"use_reloader": False, "port": port}).start()

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 5 endpoints over a single ngrok agent session.\nThe endpoints already running on this session are:\ntn_365ecj0bwiLHx6Ul332aB4yD0Vu, tn_365fb28ZZxEciAnEJUhIz7BEXwe, tn_365fkeNq6VEDkuy3p4Cu0rF8o43, tn_365ft1cShEiPynDWdlpVxq3hOGs, tn_365jbUTBaBjpPq7OHiaQKnoQQdU.\nUpgrade to a Pay-as-you-go plan at: https://dashboard.ngrok.com/billing/choose-a-plan?plan=paygo\r\n\r\nERR_NGROK_324\r\n"}}


In [7]:
import matplotlib.pyplot as plt
import io
import base64

categories = ['Apples', 'Bananas', 'Oranges', 'Grapes']
values = [20, 35, 15, 40]

plt.bar(categories, values)
plt.xlabel('Category')
plt.ylabel('Values')
plt.title('Bar Chart')

buffer = io.BytesIO()
plt.savefig(buffer, format='png')
plt.close()

image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

image_base64

'...'

In [8]:
def generate_bar_chart(categories, values):

  plt.bar(categories, values)
  plt.xlabel('Category')
  plt.ylabel('Values')
  plt.title('Bar Chart')

  buffer = io.BytesIO()
  plt.savefig(buffer, format='png')
  plt.close()

  image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

  return image_base64

In [9]:
categories = ['Apples', 'Bananas', 'Oranges', 'Grapes']
values = [20, 35, 15, 40]
generate_bar_chart(categories, values)

'...'